# Neural Network Model with Estimators

In [169]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

### Load and Prepare the Data

In [170]:
df = pd.read_csv("./filez/iris.csv")
df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0


In [171]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [172]:
# columns can't have spaces or special characters
df.columns = [
    "sepal_length",
    "sepal_width",
    "petal_length",
    "petal_width",
    "target",
]

In [173]:
# target must be integer for classification (binary-class)
df["target"] = df["target"].apply(int)
df.head(3)

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [174]:
# separate features from the target
y = df["target"]
X = df.drop("target", axis=1)

In [175]:
# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=98)

## Neural Network Model with Sequential Estimator

### Libraries:
- `Sequential`: This is a Keras model that represents a linear stack of layers.
- `Dense`: A type of layer that is fully connected, meaning each neuron in this layer is connected to all neurons in the previous layer.
- `To_categorical`: A utility function to convert integer labels into a one-hot encoded format.

In [176]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

### Convert Labels to One-Hot Encoded Format
We convert the labels into a format suitable for classification (to_categorical).

This makes each label a vector with a 1 in the position of the correct class and 0s elsewhere.

In [177]:
# Convert labels to one-hot encoded format
yc = to_categorical(y, num_classes=3)

In [178]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, yc, test_size=0.3)

### Define the Model
Define the structure of the neural network model with Sequential() -> Initializes a linear stack of layers.
- `Dense(10, input_dim=4, activation='relu')`: Adds a layer with 10 neurons, `input_dim=4` indicates the number of input features, and `relu` is the activation function.
- `Dense(20, activation='relu')`: A hidden layer with 20 neurons.
- `Dense(10, activation='relu')`: Another hidden layer with 10 neurons.
- `Dense(3, activation='softmax')`: The output layer with 3 neurons (one for each class), using `softmax` to output class probabilities.

In [179]:
model = Sequential()
model.add(Dense(10, input_dim=4, activation="relu"))  # First hidden layer
model.add(Dense(20, activation="relu"))  # Second hidden layer
model.add(Dense(10, activation="relu"))  # Third hidden layer
model.add(Dense(3, activation="softmax"))  # Output layer

### Compile the Model
Configure the model for training by setting the optimizer, loss function, and metrics.
- `optimizer='adam'`: Specifies the optimization algorithm; 'adam' is effective for a wide range of problems.
- `loss='categorical_crossentropy'`: The loss function for multi-class classification.
- `metrics=['accuracy']`: The metric to evaluate during training and testing.


In [180]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

### Train the Model
- `X_train, y_train`: The training data and labels.
- `epochs=50`: The number of times the model will work through the entire training dataset.
- `batch_size=10`: The number of samples processed before the model is updated.

In [181]:
model.fit(X_train, y_train, epochs=50, batch_size=10)

Epoch 1/50


11/11 [==============================] - 0s 710us/step - loss: 1.0679 - accuracy: 0.3714
Epoch 2/50
11/11 [==============================] - 0s 674us/step - loss: 1.0078 - accuracy: 0.4381
Epoch 3/50
11/11 [==============================] - 0s 602us/step - loss: 0.9755 - accuracy: 0.3619
Epoch 4/50
11/11 [==============================] - 0s 655us/step - loss: 0.9455 - accuracy: 0.3619
Epoch 5/50
11/11 [==============================] - 0s 636us/step - loss: 0.9151 - accuracy: 0.4000
Epoch 6/50
11/11 [==============================] - 0s 618us/step - loss: 0.8785 - accuracy: 0.5143
Epoch 7/50
11/11 [==============================] - 0s 587us/step - loss: 0.8265 - accuracy: 0.6857
Epoch 8/50
11/11 [==============================] - 0s 606us/step - loss: 0.7498 - accuracy: 0.6667
Epoch 9/50
11/11 [==============================] - 0s 573us/step - loss: 0.6934 - accuracy: 0.6381
Epoch 10/50
11/11 [==============================] - 0s 575us/step - loss: 0.6395 - accuracy: 0.6571
Epoch 11/5

### Evaluate the Model
Assess the model's performance on unseen data and to understand its accuracy and where it might be making errors.
- `y_pred` = model.predict(X_test): Uses the trained model to make predictions on the test set.
- `tf.argmax(y_pred, axis=1)`: Converts predicted probabilities to class labels.

In [182]:
y_pred = model.predict(X_test)
y_pred_classes = tf.argmax(y_pred, axis=1)
y_test_classes = tf.argmax(y_test, axis=1)

print(confusion_matrix(y_test_classes, y_pred_classes))
print(classification_report(y_test_classes, y_pred_classes))

2/2 [==============================] - 0s 1ms/step
[[21  0  0]
 [ 0 11  1]
 [ 0  2 10]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       0.85      0.92      0.88        12
           2       0.91      0.83      0.87        12

    accuracy                           0.93        45
   macro avg       0.92      0.92      0.92        45
weighted avg       0.93      0.93      0.93        45



## Legacy Neural Network Model with DNNClassifier estimator
*from TensorFlow v1.x*

In [183]:
feat_cols = [tf.feature_column.numeric_column(col) for col in X.columns]

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=98)

train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    x=X_train, y=y_train, batch_size=10, num_epochs=5, shuffle=True
)
test_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    x=X_test, y=y_test, batch_size=10, shuffle=False
)

classifier = tf.compat.v1.estimator.DNNClassifier(
    hidden_units=[10, 20, 10], n_classes=3, feature_columns=feat_cols
)
classifier.train(input_fn=train_input_fn, steps=50)

predictions = list(classifier.predict(input_fn=test_input_fn))
final_preds = [pred["class_ids"][0] for pred in predictions]

print(confusion_matrix(y_test, final_preds))
print(classification_report(y_test, final_preds))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/1d/ty_knwmj61b4d3qs091tfglm0000gn/T/tmpnu8oxron', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INF